In [1]:
from custom_dataset import Dataset_Assay, Dataset_Assay_Survival, custom_collate_assay, custom_collate_assay_survival
from models import *
from saveAndLoad import *

# LOAD DATA
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)
data_emb = labeled_data[23]
print(data_emb)
data_df_emb = pd.read_csv(data_dir+data_emb)

nlabels = len(data_df_emb['int_label'].unique())
print('n labels:',nlabels)
device = 'cuda:0'

mut_embeddings = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
ref_embeddings = np.load('../aa/canonical_ref_embeddings_esm2.npy')
tumors = pickleLoad('../aa/tumors.pkl')
assays = pickleLoad('../aa/assays.pkl')

/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


37 	 BINARYdata_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
24 	 BINARYdata_CANCER_TYPE_0MinMutations_169MinCancerType.csv
1 	 BINARYdata_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
32 	 BINARYdata_CANCER_TYPE_3MinMutations_169MinCancerType.csv
12 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
26 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
10 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
30 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_169MinCancerType.csv
23 	 data_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
25 	 data_CANCER_TYPE_0MinMutations_169MinCancerType.csv
41 	 data_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
17 	 data_CANCER_TYPE_3MinMutations_169MinCancerType.csv
11 	 data_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
28 	 data_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
4 	 data_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
47 	 data_CANCER_TYPE_DETAIL

In [6]:

class Config_Att:
    n_layer: int = 3
    input_dim: int = 24
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 1448
    num_heads: int = 8
    position_embedding: bool = False

config = Config_Att()

In [7]:
m = Classifier(config)

x = torch.rand(2,3,24)
z = x.clone()
z = z[1:,:2,:]
# x[1,2,:] = -np.inf
x[1,2,:] = 0
print(x)
print(z)

tensor([[[0.7493, 0.7543, 0.9385, 0.0438, 0.1945, 0.1059, 0.6767, 0.9847,
          0.6411, 0.9056, 0.9485, 0.7209, 0.6340, 0.6995, 0.7742, 0.5481,
          0.8686, 0.8994, 0.0191, 0.2085, 0.2336, 0.1518, 0.4591, 0.6623],
         [0.6754, 0.1534, 0.3529, 0.2411, 0.7717, 0.1295, 0.7506, 0.8238,
          0.4338, 0.0448, 0.3954, 0.2458, 0.5832, 0.0961, 0.4836, 0.2114,
          0.4941, 0.7397, 0.1926, 0.7188, 0.1100, 0.7225, 0.4451, 0.1314],
         [0.1127, 0.2534, 0.7822, 0.0797, 0.2198, 0.5985, 0.8100, 0.5226,
          0.8367, 0.3718, 0.0848, 0.8040, 0.1499, 0.7779, 0.5950, 0.9258,
          0.7869, 0.1453, 0.8146, 0.9253, 0.4937, 0.0334, 0.0407, 0.2664]],

        [[0.2694, 0.9956, 0.8311, 0.1299, 0.7045, 0.8071, 0.7862, 0.1433,
          0.1858, 0.1278, 0.2128, 0.2406, 0.0393, 0.1712, 0.0081, 0.2512,
          0.9730, 0.9814, 0.2019, 0.7042, 0.3118, 0.9808, 0.1574, 0.8423],
         [0.2291, 0.3339, 0.0252, 0.7344, 0.6192, 0.8402, 0.7559, 0.8946,
          0.9787, 0.5569, 0.8546

In [8]:
m(x)

tensor([[-0.0898, -0.0906,  0.0296,  0.0329,  0.0382,  0.0092,  0.0454,  0.0352,
          0.0267,  0.0282, -0.0405, -0.0119, -0.0020, -0.0049, -0.0198,  0.0422,
         -0.0069],
        [-0.0798, -0.1050,  0.0501,  0.0055, -0.0166,  0.0450,  0.0053,  0.0319,
          0.0203,  0.0310, -0.0483, -0.0395, -0.0432, -0.0269, -0.0195,  0.0432,
         -0.0067]], grad_fn=<AddmmBackward0>)

In [9]:
print(z)

tensor([[[0.2694, 0.9956, 0.8311, 0.1299, 0.7045, 0.8071, 0.7862, 0.1433,
          0.1858, 0.1278, 0.2128, 0.2406, 0.0393, 0.1712, 0.0081, 0.2512,
          0.9730, 0.9814, 0.2019, 0.7042, 0.3118, 0.9808, 0.1574, 0.8423],
         [0.2291, 0.3339, 0.0252, 0.7344, 0.6192, 0.8402, 0.7559, 0.8946,
          0.9787, 0.5569, 0.8546, 0.3635, 0.6113, 0.3674, 0.5550, 0.5899,
          0.4076, 0.2248, 0.3793, 0.3427, 0.6354, 0.9191, 0.0249, 0.5666]]])


In [10]:
m(z)

tensor([[-0.0798, -0.1050,  0.0501,  0.0055, -0.0166,  0.0450,  0.0053,  0.0319,
          0.0203,  0.0310, -0.0483, -0.0395, -0.0432, -0.0269, -0.0195,  0.0432,
         -0.0067]], grad_fn=<AddmmBackward0>)